<a href="https://colab.research.google.com/github/guohuanguohuan/Translation-Model/blob/main/RNN%E7%BF%BB%E8%AF%91%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python /content/drive/MyDrive/deep_learning/RNN翻译模型/inference.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/deep_learning/RNN翻译模型/inference.py", line 3, in <module>
    from translator import Seq2Seq, Encoder, Decoder, Attention 
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'translator'


In [ ]:
# 先运行以下命令训练 BPE 模型（只需运行一次）来分别生成英文和中文的词典：
# --character_coverage参数是覆盖多少用字符集，因为英文单个字符有限，所以我们设置为1.0。但是中文有很多生僻字，所以我们设置为0.9995防止词表被大量生僻词占用。

# vocab_size=16000参数是设置词表的大小，我们都设置为16000。

# 因为英语基本字符有限，中文基本字符较多，字符组合可能较多，需要分别统计频率，所以BPE生成中文词表过程会比较慢。这个过程可能需要几十分钟。建议你耐心等它生成完成。

# 如果你真的心急，可以通过下边的参数来采样一百万句子来生成词表。

# --input_sentence_size=1000000 --shuffle_input_sentence=true
import sentencepiece as spm


# spm.SentencePieceTrainer.Train('--input="/content/drive/MyDrive/deep_learning/RNN翻译模型/官方数据集/train_en.txt" --model_prefix=/content/drive/MyDrive/deep_learning/RNN翻译模型/train_en_bpe --vocab_size=16000 --model_type=bpe --character_coverage=1.0 --unk_id=0 --pad_id=1 --bos_id=2 --eos_id=3')
spm.SentencePieceTrainer.Train('--input="/content/drive/MyDrive/deep_learning/RNN翻译模型/官方数据集/train_zh.txt" --model_prefix=/content/drive/MyDrive/deep_learning/RNN翻译模型/train_zh_bpe --vocab_size=16000 --model_type=bpe --character_coverage=0.9995 --unk_id=0 --pad_id=1 --bos_id=2 --eos_id=3  --input_sentence_size=1000000 --shuffle_input_sentence=true')

In [ ]:
#训练

import os
import sentencepiece as spm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# ---------------------#
# 1. BPE Tokenization (SentencePiece)
# ---------------------#
# 先运行以下命令训练 BPE 模型（只需运行一次）：
# spm.SentencePieceTrainer.Train('--input="data\\en2cn\\train_en.txt" --model_prefix=en_bpe --vocab_size=16000 --model_type=bpe --character_coverage=1.0 --unk_id=0 --pad_id=1 --bos_id=2 --eos_id=3')
# spm.SentencePieceTrainer.Train('--input="data\\en2cn\\train_zh.txt" --model_prefix=zh_bpe --vocab_size=16000 --model_type=bpe --character_coverage=0.9995 --unk_id=0 --pad_id=1 --bos_id=2 --eos_id=3')

sp_en = spm.SentencePieceProcessor()
sp_en.load('/content/drive/MyDrive/deep_learning/RNN翻译模型/train_en_bpe.model')
sp_cn = spm.SentencePieceProcessor()
sp_cn.load('/content/drive/MyDrive/deep_learning/RNN翻译模型/train_zh_bpe.model')


def tokenize_en(text):
    return sp_en.encode(text, out_type=int)


def tokenize_cn(text):
    return sp_cn.encode(text, out_type=int)

# 中文和英文一致,取英文。
PAD_ID = sp_en.pad_id()  # 1
UNK_ID = sp_en.unk_id()  # 0
BOS_ID = sp_en.bos_id()  # 2
EOS_ID = sp_en.eos_id()  # 3


# ---------------------#
# 2. Dataset & DataLoader
# ---------------------#
class TranslationDataset(Dataset):
    ## 初始化方法，读取英文和中文训练文本。然后给每个句子前后增加<bos>和<eos>。 为了防止训练时显存不足，对于长度超过限制的
    ## 句子进行过滤。
    def __init__(self, src_file, trg_file, src_tokenizer, trg_tokenizer, max_len=10):
        with open(src_file, encoding='utf-8') as f:
            src_lines = f.read().splitlines()
        with open(trg_file, encoding='utf-8') as f:
            trg_lines = f.read().splitlines()
        assert len(src_lines) == len(trg_lines)
        self.pairs = []
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = trg_tokenizer

        for src, trg in zip(src_lines, trg_lines):
            # 每个句子前边增加<bos>后边增加<eos>
            src_ids = [BOS_ID] + self.src_tokenizer(src) + [EOS_ID]
            trg_ids = [BOS_ID] + self.trg_tokenizer(trg) + [EOS_ID]
            # 只保留输入和输出序列token数同时小于max_len的训练样本。
            if len(src_ids) <= max_len and len(trg_ids) <= max_len:
                self.pairs.append((src_ids, trg_ids))  # <-- 直接保存token id序列

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        src_ids, trg_ids = self.pairs[idx]
        return torch.LongTensor(src_ids), torch.LongTensor(trg_ids)

    ## 对一个batch的输入和输出token序列，依照最长的序列长度，用<pad> token进行填充，确保一个batch的数据形状一致，组成一个tensor。
    @staticmethod
    def collate_fn(batch):
        src_batch, trg_batch = zip(*batch)
        src_lens = [len(x) for x in src_batch]
        trg_lens = [len(x) for x in trg_batch]
        src_pad = nn.utils.rnn.pad_sequence(src_batch, padding_value=PAD_ID)
        trg_pad = nn.utils.rnn.pad_sequence(trg_batch, padding_value=PAD_ID)
        return src_pad, trg_pad, src_lens, trg_lens


# ---------------------#
# 3. Model Definitions with Attention
# ---------------------#
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        #第一层输入维度为Encoder的输出隐状态（因为是双向的，所以维度为hid_dim*2,）和Decoder的输入隐状态（单向，维度为hid_dim—）的拼接。
        self.attn = nn.Linear(hid_dim * 2 + hid_dim, hid_dim)
        #输出一个代表注意力的logit值。
        self.v = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs, mask):
        # 调整Decoder当前时间步输入隐状态的维度: [1, batch, hid_dim] -> [batch, 1, hid_dim]
        hidden = hidden.permute(1, 0, 2)
        # 调整encoder各个时间步输出隐状态的维度： [src_len, batch, hid_dim*2] -> [batch, src_len, hid_dim*2]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        src_len = encoder_outputs.shape[1]
        # 中文当前一个token需要和英文所有token计算注意力。所以需要把中文token的状态复制多份，以便进行统一拼接。
        # 因为Decoder只有当前时间步输入的隐状态，复制到和Encoder输出隐状态同样的src_len。
        hidden = hidden.repeat(1, src_len, 1)  # [batch, src_len, hid_dim]

        # 拼接Decoder当前输入的隐状态和Encoder在各个时间步输出的隐状态，然后经过一个线性层，tanh激活。
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch, src_len, hid_dim]
        # 输出Decoder当前中文token与所有英文token的注意力值。
        attention = self.v(energy).squeeze(2)  # [batch, src_len]
        # mask标志哪些位置为<pad>,对于填充的位置，注意力值为一个大的负值。这样经过softmax就为0。
        attention = attention.masked_fill(mask == 0, -1e10)
        # 利用softmax将注意力的值归一化。让生成当前中文token对输入的英文各个token的注意力之和为1。
        return torch.softmax(attention, dim=1)  # [batch, src_len]


class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, n_layers=3):
        super().__init__()
        # 默认3个循环层
        self.n_layers=n_layers
        # 定义Embedding,可以将BPE分词输出的token id，转化为emd_dim的embedding向量。<pad>不参与运算，它的embedding不需要学习。
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=PAD_ID)
        # 定义双向LSTM模型
        self.bi_lstm = nn.LSTM(emb_dim, hid_dim, n_layers, bidirectional=True)
        # 定义线性层列表来降低维度。因为每个Encoder是双向的，隐状态和细胞状态为hid_dim*2，Decoder是单向的，隐状态和细胞状态维度为hid_dim。
        self.fc_hidden = nn.ModuleList([nn.Linear(hid_dim * 2, hid_dim) for _ in range(n_layers)])
        self.fc_cell = nn.ModuleList([nn.Linear(hid_dim * 2, hid_dim) for _ in range(n_layers)])

    def forward(self, src, src_len):
        embedded = self.embedding(src)
        #将一个 padded sequence（已经填充到统一长度的 batch 序列） 转换为一个特殊的 PackedSequence 对象
        #这个对象在传入 RNN 时能跳过 padding 部分的计算。
        packed = nn.utils.rnn.pack_padded_sequence(embedded, src_len, enforce_sorted=False)
        #outputs,形状为 (seq_len, batch_size, hid_dim*2),表示每个时间步、最后一层LSTM的双向隐状态拼接。
        #(hidden, cell) ，形状都为(num_layers * 2, batch_size, hid_dim)表示每一层、每个方向在最后一个时间步的隐状态或细胞状态。
        outputs, (hidden, cell) = self.bi_lstm(packed)

        #将 PackedSequence 类型的输出还原成带 padding 的标准 Tensor，方便后续处理。
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)  # [src_len, batch, hid_dim*2]

        # 重塑隐藏状态和细胞状态: [n_layers * 2, batch, hid_dim] -> [n_layers, 2, batch, hid_dim]
        hidden = hidden.view(self.n_layers, 2, -1, hidden.size(2))
        cell = cell.view(self.n_layers, 2, -1, cell.size(2))

        # 为每一层处理前向和后向状态
        final_hidden = []
        final_cell = []

        for layer in range(self.n_layers):
            # 对LSTM每一层最后一个时间步，将隐状态的正向和反向状态合并，细胞状态的正向和反向状态合并，分别通过一个线性层将维度从hid_dim*2降低为hid_dim维度。
            h_cat = torch.cat((hidden[layer][-2], hidden[layer][-1]), dim=1)
            c_cat = torch.cat((cell[layer][-2], cell[layer][-1]), dim=1)
            h_layer = torch.tanh(self.fc_hidden[layer](h_cat)).unsqueeze(0)
            c_layer = torch.tanh(self.fc_cell[layer](c_cat)).unsqueeze(0)

            final_hidden.append(h_layer)
            final_cell.append(c_layer)
        # 调整好维度为hid_dim的隐状态和细胞状态，可以传递给Decoder。
        hidden_concat = torch.cat(final_hidden, dim=0)
        cell_concat = torch.cat(final_cell, dim=0)
        return outputs, hidden_concat, cell_concat


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention, n_layers=3):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=PAD_ID)
        # 单向LSTM，输入维度为注意力加权后的，Encoder输出隐状态维度（hid_dim*2）加上输入token的embedding的维度emb_dim。
        self.rnn = nn.LSTM(hid_dim * 2 + emb_dim, hid_dim,num_layers=n_layers)
        # 定义最终分类头，输入为3倍的hid_dim，输出为字典大小。
        # 为什么输入是3倍的hid_dim呢？
        # 因为预测中文token时，输入不光是Decoder的最后一层的隐状态，还拼接了注意力向量。
        # 这样对这个时间步生成翻译的中文token会有帮助。而注意力向量来自双向的Encoder，所以要额外加上一个2倍的hid_dim.
        self.fc_out = nn.Linear(hid_dim*3, output_dim)

    def forward(self, input_token, hidden, cell, encoder_outputs, mask):
        # input_token: [batch]
        # 输入单个字符，增加一个维度
        input_token = input_token.unsqueeze(0)  # [1, batch]
        # 获取token的embedding
        embedded = self.embedding(input_token)  # [1, batch, emb_dim]
        # 获取当前步的输入隐状态
        last_hidden = hidden[-1].unsqueeze(0)
        # 当前步对所有encoder输出的注意力
        a = self.attention(last_hidden, encoder_outputs, mask)  # [batch, src_len]
        a = a.unsqueeze(1)  # [batch, 1, src_len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # [batch, src_len, enc_hid_dim*2]

        # 用矩阵乘法获取注意力上下文向量
        weighted = torch.bmm(a, encoder_outputs)  # [batch, 1, enc_hid_dim*2]
        weighted = weighted.permute(1, 0, 2)  # [1, batch, enc_hid_dim*2]
        # 拼接输入token编码向量和注意力上下文向量
        lstm_input = torch.cat((embedded, weighted), dim=2)  # [1, batch, emb_dim + enc_hid_dim*2]
        # 一次只执行lstm的一个时间步。
        output, (hidden, cell) = self.rnn(lstm_input, (hidden, cell))  # output: [1, batch, hid_dim]

        # 移除第0维（第一个维度）
        output = output.squeeze(0)  # [batch, hid_dim]
        weighted = weighted.squeeze(0)
        # 拼接输出和注意力上下文向量，进入分类头，计算分类logtis
        prediction = self.fc_out(torch.cat((output, weighted), dim=1))  # [batch, output_dim]

        return prediction, hidden, cell, a.squeeze(1)  # attention weights for visualization



class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_len, trg):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, vocab_size).to(self.device)
        # 调用encoder
        encoder_outputs, hidden, cell = self.encoder(src, src_len)

        input_token = trg[0]
        mask = (src != PAD_ID).permute(1, 0)
        # 逐步调用Decoder
        for t in range(1, max_len):
            output, hidden, cell, _ = self.decoder(input_token, hidden, cell, encoder_outputs, mask)
            outputs[t] = output
            input_token = trg[t]

        return outputs


# ---------------------#
# 4. Training
# ---------------------#
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    step_loss = 0  # 用于累计每个step的loss
    step_count = 0  # 当前step计数器

    for i, (src, trg, src_len, _) in enumerate(iterator):
        src, trg = src.to(model.device), trg.to(model.device)
        optimizer.zero_grad()
        output = model(src, src_len, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

        # 更新损失统计
        step_loss += loss.item()
        epoch_loss += loss.item()
        step_count += 1

        # 每100个step打印一次
        if (i + 1) % 100 == 0:
            avg_step_loss = step_loss / step_count
            print(f'Step [{i + 1}/{len(iterator)}] | Loss: {avg_step_loss:.4f}')
            step_loss = 0  # 重置step损失
            step_count = 0  # 重置step计数器

    return epoch_loss / len(iterator)  # 返回整个epoch的平均loss


# ---------------------#
# 5. Main
# ---------------------#
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = TranslationDataset('/content/drive/MyDrive/deep_learning/RNN翻译模型/官方数据集/train_en.txt', '/content/drive/MyDrive/deep_learning/RNN翻译模型/官方数据集/train_zh.txt', tokenize_en, tokenize_cn)
    loader = DataLoader(dataset, batch_size=512, shuffle=True, collate_fn=TranslationDataset.collate_fn)

    INPUT_DIM = sp_en.get_piece_size()
    OUTPUT_DIM = sp_cn.get_piece_size()
    ENC_EMB_DIM = 16
    DEC_EMB_DIM = 16
    HID_DIM = 16

    attention = Attention(HID_DIM).to(device)
    encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM).to(device)
    decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, attention).to(device)
    model = Seq2Seq(encoder, decoder, device).to(device)
    torch.save(model.state_dict(), '/content/drive/MyDrive/deep_learning/RNN翻译模型/seq2seq_bpe_attention.pt')
    model.load_state_dict(torch.load('/content/drive/MyDrive/deep_learning/RNN翻译模型/seq2seq_bpe_attention.pt', map_location=device))
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)

    N_EPOCHS = 1

    for epoch in range(N_EPOCHS):
        loss = train(model, loader, optimizer, criterion)
        print(f'Epoch {epoch + 1}/{N_EPOCHS} | Loss: {loss:.4f}')
        torch.save(model.state_dict(), '/content/drive/MyDrive/deep_learning/RNN翻译模型/seq2seq_bpe_attention.pt')

Step [100/4631] | Loss: 8.3030
Step [200/4631] | Loss: 6.5841
Step [300/4631] | Loss: 6.4356
Step [400/4631] | Loss: 6.3379
Step [500/4631] | Loss: 6.2502
Step [600/4631] | Loss: 6.1812
Step [700/4631] | Loss: 6.0904
Step [800/4631] | Loss: 6.0400
Step [900/4631] | Loss: 6.0070
Step [1000/4631] | Loss: 5.9694
Step [1100/4631] | Loss: 5.9324
Step [1200/4631] | Loss: 5.8755
Step [1300/4631] | Loss: 5.8068
Step [1400/4631] | Loss: 5.7382
Step [1500/4631] | Loss: 5.7017
Step [1600/4631] | Loss: 5.6795
Step [1700/4631] | Loss: 5.6551
Step [1800/4631] | Loss: 5.6366
Step [1900/4631] | Loss: 5.6121
Step [2000/4631] | Loss: 5.6061
Step [2100/4631] | Loss: 5.5736
Step [2200/4631] | Loss: 5.5601
Step [2300/4631] | Loss: 5.5454
Step [2400/4631] | Loss: 5.5230
Step [2500/4631] | Loss: 5.5080
Step [2600/4631] | Loss: 5.4912
Step [2700/4631] | Loss: 5.4747
Step [2800/4631] | Loss: 5.4496
Step [2900/4631] | Loss: 5.4380
Step [3000/4631] | Loss: 5.4095
Step [3100/4631] | Loss: 5.4007
Step [3200/4631] 

In [ ]:
#人工测试

import torch
import sentencepiece as spm
# from translator import Seq2Seq, Encoder, Decoder, Attention

# ---------------------#
# 1. Load Tokenizers
# ---------------------#
sp_en = spm.SentencePieceProcessor()
sp_en.load('/content/drive/MyDrive/deep_learning/RNN翻译模型/train_en_bpe.model')
sp_cn = spm.SentencePieceProcessor()
sp_cn.load('/content/drive/MyDrive/deep_learning/RNN翻译模型/train_zh_bpe.model')
PAD_ID = sp_en.pad_id()
BOS_ID = sp_en.bos_id()
EOS_ID = sp_en.eos_id()

# ---------------------#
# 2. Load Trained Model
# ---------------------#
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model hyperparameters (must match training)
INPUT_DIM = sp_en.get_piece_size()
OUTPUT_DIM = sp_cn.get_piece_size()
ENC_EMB_DIM = 16
DEC_EMB_DIM = 16
HID_DIM = 16
N_LAYERS = 3

attention = Attention(HID_DIM).to(DEVICE)
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, n_layers=N_LAYERS).to(DEVICE)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, attention, n_layers=N_LAYERS).to(DEVICE)
model = Seq2Seq(encoder, decoder, DEVICE).to(DEVICE)
model.load_state_dict(torch.load('/content/drive/MyDrive/deep_learning/RNN翻译模型/seq2seq_bpe_attention.pt', map_location=DEVICE))
model.eval()

# ---------------------#
# 3. Translation Function (Greedy)
# ---------------------#
def translate_sentence(sentence, max_len=10):
    # Tokenize and convert to IDs
    tokens = [BOS_ID] + sp_en.encode(sentence, out_type=int) + [EOS_ID]
    src_tensor = torch.LongTensor(tokens).unsqueeze(1).to(DEVICE)  # [src_len, 1]
    src_len = [len(tokens)]

    # 调用Encoder
    with torch.no_grad():
        encoder_outputs, hidden, cell = encoder(src_tensor, src_len)

    # 第一个输入token，序列起始token：<bos>
    trg_indices = [BOS_ID]
    # 逐个token，循环调用Decoder。
    for _ in range(max_len):
        # 最新生成的token作为输入
        trg_tensor = torch.LongTensor([trg_indices[-1]]).to(DEVICE)
        with torch.no_grad():
            output, hidden, cell, _ = decoder(trg_tensor, hidden, cell, encoder_outputs,
                                               (src_tensor != PAD_ID).permute(1, 0))
        # 取预测概率最大的token作为输出
        pred_token = output.argmax(1).item()
        trg_indices.append(pred_token)
        if pred_token == EOS_ID:
            break

    # 将token id解码为文字 (跳过<bos>和<eos>)
    translated = sp_cn.decode(trg_indices[1:-1])
    return translated

# ---------------------#
# 4. Interactive Test
# ---------------------#
if __name__ == '__main__':
    while True:
        src_sent = input("Enter English sentence (or 'quit' to exit): ")
        if src_sent.lower() in ['quit', 'exit']:
            break
        translation = translate_sentence(src_sent)
        print(f"Chinese Translation: {translation}\n")

Enter English sentence (or 'quit' to exit): hello
Chinese Translation: ,

Enter English sentence (or 'quit' to exit): hi
Chinese Translation: ...

Enter English sentence (or 'quit' to exit): what
Chinese Translation: ...

Enter English sentence (or 'quit' to exit): nihao
Chinese Translation: 在人



KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
#BLEU评价指标

import sacrebleu
# from inference import translate_sentence
# 读取验证集的英文原文和中文参考
with open('/content/drive/MyDrive/deep_learning/RNN翻译模型/官方数据集/valid_en.txt', 'r', encoding='utf-8') as f:
    src_sentences = [line.strip() for line in f.readlines()]

with open('/content/drive/MyDrive/deep_learning/RNN翻译模型/官方数据集/valid_zh.txt', 'r', encoding='utf-8') as f:
    ref_sentences = [line.strip() for line in f.readlines()]

# 检查长度是否匹配
assert len(src_sentences) == len(ref_sentences), "源语言和参考翻译句子数不匹配"

# 用模型生成翻译
hypotheses = []
for i, src in enumerate(src_sentences):
    print(f"Translating {i+1}/{len(src_sentences)}...")
    translation = translate_sentence(src)
    print(ref_sentences[i], translation)
    hypotheses.append(translation.strip())

# 计算 BLEU
bleu = sacrebleu.corpus_bleu(hypotheses, [ref_sentences], tokenize='zh')

print("\n========== BLEU Evaluation Result ==========")
print(f"BLEU Score: {bleu.score:.2f}")

流式输出内容被截断，只能显示最后 5000 行内容。
是，屁股裸露频道。你喜欢露屁股的吗？ 嘿,我们,我们吗?
Translating 5503/8000...
他曾默默无闻想靠杀人出名但人们不会让他得逞的。 他被被被被人。
Translating 5504/8000...
我觉得你是我所见过的最美丽的女人。 我有有的的。
Translating 5505/8000...
如果我留下，待遇必须要胜过这个。 但,我有有了。
Translating 5506/8000...
我来这儿是因为皮特让我装一点货。 我想你的你的了。
Translating 5507/8000...
为了能为他们创造这一点点宝贵的时间成千上万的士兵们牺牲了性命。 他被被人的的。
Translating 5508/8000...
从表面来看还和我们的脚印很相似， 因为有有有人的的的
Translating 5509/8000...
从埃及弄个小牛头骨到底要多久？ 我们在到的的的
Translating 5510/8000...
上帝知道我爱他，但这是个摆脱了他的夜晚。为了孩子。 我知道。我我。
Translating 5511/8000...
但是他们的小面包是撒过芝麻的。我的小面包就没撒芝麻。 她。她她了。
Translating 5512/8000...
嗯，这是一张医务床，你知道，用来生孩子的。 但,我,我,
Translating 5513/8000...
把削减工资时砍掉的几百亿暂时归还。 大大大大的。
Translating 5514/8000...
我喜欢这个包，喜欢鞋，喜欢一切。我很乐意留下。 ,我们他的。
Translating 5515/8000...
我那天花了一个小时跟老师谈话， 但有有的的,
Translating 5516/8000...
我们的无障碍特大床房提供1张特大床。 大大大大的。
Translating 5517/8000...
是的。呃，我们也不想惹恼整个墓地的死人， 好吧,我们,我们,
Translating 5518/8000...
我不愿意受威胁。那十个人也一样。 。我们有个。
Translating 5519/8000...
帮我个忙，好吗？你能不能 你,你你你你你
Translating 5520/8000...
准备好掌声鼓声也请准备好。 你你的